# Check non-Payroll Operating
Each agency has a non-payroll operating cost associated with it. 

## Set up environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import os
import sys
import numpy as np

In [3]:
sys.path.insert(0, os.path.abspath("../.."))

## Get Agencies

In [4]:
from Initialize_Agencies import get_agencies

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Cost_Type_Code/Pensions/Statewide_Pensions.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [5]:
yr = list(range(2016,2020))
agencies = get_agencies(yr)

/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes/Agency_Classes_Big.py:93: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.get_expenditures_by_year()


got here for CPCS


/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes/Agency_Classes_Big.py:93: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.get_expenditures_by_year()


## Save to .csv

In [6]:
out_df = pd.DataFrame(columns=yr)
for _, agency in agencies.items():
    out_df.loc[agency.official_name,:] = agency.non_payroll_operating_expenditures_by_year

In [7]:
display(out_df)

,2016,2017,2018,2019
TRIAL COURT (TRC),111466652.300,108356313.990,110473319.370,124205832.610
COMMITTEE FOR PUBLIC COUNSEL SERVICES (CPC),10797600.960,11650922.760,11448343.740,14652018.410
DEPARTMENT OF CORRECTION (DOC),97725853.440,95048173.120,96505205.600,98605034.890
SUFFOLK DISTRICT ATTORNEY (SUF),2736672.790,2795898.000,2929760.250,3485928.940
SHERIFF DEPARTMENT SUFFOLK (SDS),12144514.950,10874851.970,11296755.810,12219683.860
DEPARTMENT OF STATE POLICE (POL),31973742.890,28617888.560,32748216.500,35268842.600
MASSACHUSETTS BAY TRANSPORTATION AUTHORITY (MBT),0,0,0,0
DISTRICT ATTORNEY ASSOCIATION (DAA),2768464.740,2751520.930,2679983.480,2774395.860
SHERIFFS DEPARTMENT ASSOCIATION (SDA),114062.070,89715.150,82743.850,79547.060
PAROLE BOARD (PAR),1499617.290,1320663.170,1388778.360,1707042.470


In [8]:
# out_df.to_csv("Non_Payroll_Final_Dec17.csv")

## Check state Agencies
### Get df of all gov spending for 2018

In [9]:
from sodapy import Socrata

In [10]:
app_token = "2Qa1WiG8G4kj1vGVd2noK7zP0"
client = Socrata("cthru.data.socrata.com", app_token)
client.timeout = 40


In [1]:
all_expenditures = pd.DataFrame(client.get("pegc-naaa", where = "budget_fiscal_year=2018", limit=9999999))

NameError: name 'pd' is not defined

In [ ]:
all_expenditures.shape

## Apply methodology and get it to look like out_df

Get rid of expenditures on payroll vendors

In [ ]:
all_expenditures = all_expenditures[all_expenditures["vendor"].str.contains("PAYROLL")==False]

In [ ]:
all_expenditures.shape

Get rid of federal, intragovernmental, and capital appropriations

In [ ]:
all_expenditures["appropriation_type"]

In [ ]:
all_expenditures = all_expenditures[(all_expenditures["appropriation_type"].str.contains("FEDERAL")== False)&
                                    (all_expenditures["appropriation_type"].str.contains("INTRAGOVERNMENTAL")==False)&
                                    (all_expenditures["appropriation_type"].str.contains("CAPITAL")==False)]
all_expenditures.shape

In [ ]:
all_expenditures = all_expenditures[all_expenditures["object_class"].str.contains("DD") == False]

In [ ]:
all_expenditures = all_expenditures[all_expenditures["object_code"].str.contains("R24")==False]
all_expenditures.shape

In [ ]:
all_expenditures.loc["amount"] = all_expenditures["amount"].astype(float)

In [ ]:
all_expenditures.head(5)

In [ ]:
by_dept = all_expenditures.groupby("department").sum()["amount"]

In [ ]:
out_df

In [ ]:
state_agency_names = list(out_df.index[:-4])

In [ ]:
state_agency_names.remove("MASSACHUSETTS BAY TRANSPORTATION AUTHORITY (MBT)")

In [ ]:
import numpy as np

In [ ]:
assert np.isclose(by_dept[state_agency_names].values, out_df.loc[state_agency_names, 2018].astype(float).values)

## Local Police

### Boston
In Boston budget documents, spending is broken down into operating budget and external funds budget. All of operating budget counts towards final analysis, plus external funds deemed to be from state agencies (so not federal and not private). 
Both operating budget and external funds are broken down into categories like "Personnel Services", "Contractual Services", etc. 
Personnel services don't count towards this category. Niether does worker's comp medical  
For Boston, this is non-personnel services expenditures from plus select external funds. The external funds included are non-federal and non-private.

Look at 2017 as example, so 2019 budget pdf

To get non-payroll, non-benefit dollars from operating budget, take total expenditures and subtract payroll and worker's comp medical

In [ ]:
total_2017_operating = 364594820
personnel_2017 = 332157566
workers_comp_2017 = 132926
non_payroll_pre_external = total_2017_operating - personnel_2017 - workers_comp_2017
print("before including external funds: ", non_payroll_pre_external)
print("this should be around", (non_payroll_pre_external/out_df.loc["Boston PD", 2017])*100, "% of total non-payroll operating")
print("there should be another", (out_df.loc["Boston PD", 2017] - non_payroll_pre_external)/10**6, "million dollars in external funds")

Ther are around 10 mil in total in external funds in 2017 so this seems plausible

In [ ]:
external_personnel = 3919606
external_total = 9710199
external_funds_non_payroll = external_total - external_personnel
external_funds_non_payroll

In [ ]:
federal_external = 35080 #Academy Revolving
federal_external += 32849 #Boston multi-cultural advocacy support
federal_external += 686178 #boston reentry 
federal_external += 13012 #Canine revolving
federal_external += 624 #CHRP
federal_external += 25237 #Coverdell N.F.S.I.
federal_external += 300960 #DNA Laboratory
federal_external += 33407 #Hackney revolving
federal_external += 5888 #Injury Surveilllance Project
federal_external += 135889 #JAG equipment
federal_external += 32259 #Justice and mental health expansion
federal_external += 589129 #JAG 
federal_external += 30496 #National forum capacity building demo
federal_external += 54701 #OJJDP Youth violence prevention
federal_external += 156762 #Police fitness center revolving
federal_external += 412391 #Port Security
federal_external += 74506 #Smart policing
federal_external += 69766 # VAWA

In [ ]:
fraction_external_federal = federal_external/external_total

In [ ]:
external_non_payroll_non_federal = external_funds_non_payroll*(1-fraction_external_federal)
non_payroll_total = non_payroll_pre_external + external_non_payroll_non_federal

In [ ]:
out_df.loc["Boston PD", 2017] - non_payroll_total

In [ ]:
assert (non_payroll_total - out_df.loc["Boston PD", 2017]) < 10

Within 10 bucks good enough for me.

Last thing to look at: does federal dollars to BPD change per year? If so double check why

In [ ]:
bostonPD = agencies["Boston PD"]
bostonPD.federal_expenditures_by_year

### Chelsea

In report definitely point to how the FY20 and FY21 budgets have different numbers for this value. 2016 is higher because this line-item in FY20 budget are all higher and FY20 budget only provides most up-to-date number for this value 

In [ ]:
chelseaPD = agencies["Chelsea PD"]
chelseaPD.budget_summary

In [ ]:
chelseaPD.non_payroll_operating_expenditures_by_year[2016] = chelseaPD.budget_summary.loc["Total Budget", 2016] - \
    chelseaPD.budget_summary.loc["Payroll Budget", 2016]

In [ ]:
chelseaPD.non_payroll_operating_expenditures_by_year

### Revere

Ok great all numbers match budget documents perfectly

### Winthrop

Point to how winthrop budget document has a category for fringe benefits with one line-item of "Longevity" with all zeros for each year. What a mess 

In [ ]:
display(out_df)

In [ ]:
winthropPD = agencies["Winthrop PD"]

In [ ]:
winthropPD.budget_summary

Ok all looks good here